In [1]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from pyspark.sql.functions import when
import seaborn as sns
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.mllib.feature import StandardScaler, PCA
from pyspark.mllib.stat import Statistics
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import from_unixtime, year, lower
from pyspark.sql.functions import sum, when, col, lit, isnull
import pyspark


StatementMeta(, , , Waiting, )

### REALIZANDO JOIN YELP

In [2]:
df = spark.sql("SELECT * FROM LH_destino_PFH.BUSINESSYELP_1")


StatementMeta(, , , Waiting, )

In [3]:
y_review = spark.sql("SELECT * FROM LH_destino_PFH.REVIEWYELP_1")

StatementMeta(, , , Waiting, )

In [4]:
df.columns

StatementMeta(, , , Waiting, )

['BUSINESS_ID',
 'NAME',
 'CITY',
 'STATE',
 'LATITUDE',
 'LONGITUDE',
 'STARS',
 'REVIEW_COUNT',
 'CATEGORIES',
 'CATEGORIEPERS']

In [5]:
y_review = y_review.withColumnRenamed("BUSINESS_ID", "REVIEW_BUSINESS_ID")
df = df.drop("STARS")
y_review_business = df.join(y_review, df["BUSINESS_ID"] == y_review["REVIEW_BUSINESS_ID"], 'inner') \
    .select(df["BUSINESS_ID"], df["NAME"], df["STATE"], df["LATITUDE"], df["LONGITUDE"],
            df["REVIEW_COUNT"], df["CATEGORIES"], df["CATEGORIEPERS"], 
            y_review["REVIEW_ID"], y_review["USER_ID"], y_review["STARS"], 
            y_review["DATE"], y_review["SENTIMENT_ANALYSIS"],df["CITY"])
            
y_review_business = y_review_business.drop("CATEGORIES")
y_review_business = y_review_business.withColumnRenamed("FECHA", "DATE")
y_review_business = y_review_business.withColumnRenamed("ANALISIS_SENTIMIENTO", "SENTIMENT_ANALYSIS")
y_review_business = y_review_business.withColumnRenamed("REVIEW_COUNT", "NUM_OF_REVIEWS")
#y_review_business = y_review_business.filter(y_review_business["CATEGORIEPERS"].contains("Hotel"))
y_review_business = y_review_business.withColumnRenamed("CATEGORIEPERS", "CATEGORY")
y_review_business = y_review_business.withColumn("DATA_SOURCE", lit("YELP"))
y_review_business.columns

StatementMeta(, , , Waiting, )

['BUSINESS_ID',
 'NAME',
 'STATE',
 'LATITUDE',
 'LONGITUDE',
 'NUM_OF_REVIEWS',
 'CATEGORY',
 'REVIEW_ID',
 'USER_ID',
 'STARS',
 'DATE',
 'SENTIMENT_ANALYSIS',
 'CITY',
 'DATA_SOURCE']

## GOOGLE

In [6]:
metadata = spark.sql("SELECT * FROM LH_destino_PFH.METADATAGOOGLE_1")
metadata = metadata.drop("URL","CATEGORY","AVG_RATING","ADDRESS","CATEGORY")
# metadata = metadata.filter(metadata["CATEGORY SUMMARY"].contains("Hostel"))
reviews = spark.sql("SELECT * FROM LH_destino_PFH.ESTADOSGOOGLE_1")
reviews = reviews.withColumnRenamed("GMAP_ID", "GMAP_ID_REVIEW")
reviews = reviews.drop("NAME","TEXT")
print("numero de registros de metadata de google",metadata.count())
print("numero de registros de reseñas de google",reviews.count())

StatementMeta(, , , Waiting, )

numero de registros de metadata de google 3025011
numero de registros de reseñas de google 27619402


In [7]:
google_reviews = metadata.join(reviews, metadata.GMAP_ID == reviews.GMAP_ID_REVIEW, 'inner')
google_reviews = google_reviews.withColumnRenamed("CATEGORY SUMMARY", "CATEGORY")
google_reviews = google_reviews.withColumnRenamed("RATING", "STARS")
google_reviews = google_reviews.withColumnRenamed("GMAP_ID", "BUSINESS_ID")
google_reviews = google_reviews.drop("GMAP_ID_REVIEW")
google_reviews = google_reviews.withColumn("DATA_SOURCE", lit("GOOGLE"))
#google_reviews = google_reviews.where(google_reviews["CATEGORY"] == "Hostel")
google_reviews.columns
print("numero de registros de reseñas y metadata de google",google_reviews.count())

StatementMeta(, , , Waiting, )

numero de registros de reseñas y metadata de google 15652922


In [8]:
google_reviews = google_reviews.withColumn("CATEGORY", 
                        when(google_reviews["CATEGORY"] == "Hostel", "Hotel")
                        .otherwise(google_reviews["CATEGORY"]))
google_reviews.groupBy("CATEGORY").count().show()

StatementMeta(, , , Waiting, )

+--------------+--------+
|      CATEGORY|   count|
+--------------+--------+
|     Education|    1955|
|      Cultural|  192112|
|     Nightlife|   18491|
|        Health|   69557|
|      Religion|    9373|
|         Media|     729|
|Food and drink| 3794137|
|         Other|10907516|
|      Shopping|  426661|
|         Hotel|   20844|
|          Care|  210647|
|       Service|     870|
|          Pets|      30|
+--------------+--------+



In [9]:
google_reviews.select("STATE").show()

StatementMeta(, , , Waiting, )

+---------+
|    STATE|
+---------+
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|Louisiana|
|  Florida|
|  Florida|
|  Florida|
|  Florida|
|  Florida|
|  Florida|
|  Florida|
|  Florida|
+---------+
only showing top 20 rows



In [10]:
nulos = google_reviews.filter(isnull(google_reviews['STATE'])).count()
print("Numero de nulos Google", nulos)
google_reviews.groupBy("STATE").count().show()

StatementMeta(, , , Waiting, )

Numero de nulos Google 0


+--------------+-------+
|         STATE|  count|
+--------------+-------+
|         Texas|1156185|
|        Nevada| 466017|
|    Washington|1293136|
|      Delaware| 124036|
|       Georgia| 205902|
|      Virginia|1240347|
|North Carolina|1019310|
|      Maryland| 822711|
|       Alabama| 703801|
| Massachusetts| 798175|
|     Louisiana| 598279|
|   Mississippi| 310929|
|       Florida|2740222|
|South Carolina| 792602|
|    California|2638493|
|      New York| 742777|
+--------------+-------+



### Google correción columna STATE

In [11]:
state = spark.read.format("csv").option("header","true").load("Files/uszips.csv")
google_reviews = google_reviews.withColumn("STATE", lower(google_reviews["STATE"]))
state = state.withColumn("state_name", lower(state["state_name"]))

unique_state_id = state.select("state_name", "state_id").distinct()

# Realiza el join entre google_reviews y unique_state_id
google_reviews = google_reviews.join(unique_state_id, google_reviews.STATE == unique_state_id.state_name, 'left_outer')
# Muestra solo las columnas relevantes después del join
google_reviews = google_reviews.drop("STATE","state_name")
google_reviews = google_reviews.withColumnRenamed("state_id", "STATE")
google_reviews.select("STATE").show()

StatementMeta(, , , Waiting, )

+-----+
|STATE|
+-----+
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   LA|
|   FL|
|   FL|
|   FL|
|   FL|
|   FL|
|   FL|
|   FL|
|   FL|
+-----+
only showing top 20 rows



In [13]:
nulos = google_reviews.filter(isnull(google_reviews['STATE'])).count()
print("Numero de nulos Google", nulos)
google_reviews.groupBy("STATE").count().show()

StatementMeta(, , , Waiting, )

Numero de nulos Google 0


+-----+-------+
|STATE|  count|
+-----+-------+
|   SC| 792602|
|   LA| 598279|
|   VA|1240347|
|   NV| 466017|
|   CA|2638493|
|   NC|1019310|
|   MD| 822711|
|   DE| 124036|
|   WA|1293136|
|   MS| 310929|
|   AL| 703801|
|   NY| 742777|
|   TX|1156185|
|   GA| 205902|
|   MA| 798175|
|   FL|2740222|
+-----+-------+



## Unión de Google & Yelp

In [14]:
print("Esquema de y_reviews_business:")
y_review_business.printSchema()
print(y_review_business.count())
print("Esquema de google_reviews:")
google_reviews.printSchema()
print(google_reviews.count())


StatementMeta(, , , Waiting, )

Esquema de y_reviews_business:
root
 |-- BUSINESS_ID: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- NUM_OF_REVIEWS: long (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- REVIEW_ID: string (nullable = true)
 |-- USER_ID: string (nullable = true)
 |-- STARS: long (nullable = true)
 |-- DATE: date (nullable = true)
 |-- SENTIMENT_ANALYSIS: long (nullable = true)
 |-- CITY: string (nullable = true)
 |-- DATA_SOURCE: string (nullable = false)



221845
Esquema de google_reviews:
root
 |-- NAME: string (nullable = true)
 |-- BUSINESS_ID: string (nullable = true)
 |-- NUM_OF_REVIEWS: long (nullable = true)
 |-- CATEGORY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- USER_ID: double (nullable = true)
 |-- STARS: long (nullable = true)
 |-- DATE: date (nullable = true)
 |-- SENTIMENT_ANALYSIS: long (nullable = true)
 |-- DATA_SOURCE: string (nullable = false)
 |-- STATE: string (nullable = true)



15652922


In [20]:
columnas_comunes = set(y_review_business.columns).intersection(set(google_reviews.columns))
google_reviews = google_reviews.withColumn("CITY", lit("Valor predeterminado"))
# Reorganizar las columnas en y_review_business
columnas_y_review_business = list(columnas_comunes) + [col for col in y_review_business.columns if col not in columnas_comunes]
y_review_business = y_review_business.select(columnas_y_review_business)

# Reorganizar las columnas en google_reviews
columnas_google_reviews = list(columnas_comunes) + [col for col in google_reviews.columns if col not in columnas_comunes]
google_reviews = google_reviews.select(columnas_google_reviews)


google_reviews = google_reviews.withColumn("REVIEW_ID", lit(None).cast("string"))
google_reviews = google_reviews.withColumn("USER_ID", google_reviews["USER_ID"].cast("string"))
merged_reviews = y_review_business.union(google_reviews)


StatementMeta(, , , Waiting, )

In [21]:
merged_reviews.count()

StatementMeta(, , , Waiting, )

15874767

In [22]:
output_path = "Files/DATA_PROCESADA/merged_reviews"
merged_reviews_single_partition = merged_reviews.coalesce(1)
merged_reviews_single_partition.write.csv(output_path, mode="overwrite", header = "True")

StatementMeta(, , , Waiting, )

In [23]:
merged_reviews_single_partition.columns

StatementMeta(, , , Waiting, )

['SENTIMENT_ANALYSIS',
 'CATEGORY',
 'LATITUDE',
 'DATA_SOURCE',
 'LONGITUDE',
 'NAME',
 'USER_ID',
 'STARS',
 'DATE',
 'NUM_OF_REVIEWS',
 'STATE',
 'BUSINESS_ID',
 'REVIEW_ID',
 'CITY']

# Trabajando con la tabla merged

In [ ]:
ERROR

StatementMeta(, , , Waiting, )

In [ ]:
spark = SparkSession.builder.appName("ensayo").getOrCreate()

StatementMeta(, , , Waiting, )

In [ ]:
path = "Files/DATA_PROCESADA/df.ESTADOSGOOGLE.csv"
g_reviews = spark.read.csv(path, header = "True")

StatementMeta(, , , Waiting, )

In [ ]:
total_registros = g_reviews.count()
num_valores_entre_4_y_5 = g_reviews.filter((g_reviews["RATING"] >= 4) & (g_reviews["RATING"] <= 5)).count()
num_valores_entre_1_y_3 = g_reviews.filter((g_reviews["RATING"] >= 1) & (g_reviews["RATING"] <= 3)).count()
num_registros_en_cero = g_reviews.filter(g_reviews["RATING"] == 0).count()
cal
print("Número total de registros donde 'RATING' es igual a 0:", num_registros_en_cero)
print("Número total de registros:", total_registros)
print("Número total de valores entre 4 y 5 en 'RATING':", num_valores_entre_4_y_5)
print("Número total de valores entre 1 y 3 en 'RATING':", num_valores_entre_1_y_3)


print("Número total de registros con valores fuera del rango de 0 a 5 en 'RATING':", num_registros_fuera_de_rango)
print("Valores únicos fuera del rango de 0 a 5 en 'RATING' y su conteo:")
valores_unicos_fuera_de_rango.show()
print("Número total de valores únicos fuera del rango de 0 a 5 en 'RATING':", num_valores_unicos_fuera_de_rango)


StatementMeta(, , , Waiting, )

In [ ]:
path = "Files/DATA_PROCESADA/df_BUSINESSYELP.csv"
y_reviews = spark.read.csv(path, header = "True")

StatementMeta(, , , Waiting, )

In [ ]:
total_registros = y_reviews.count()
num_valores_entre_4_y_5 = y_reviews.filter((y_reviews["STARS"] >= 4) & (y_reviews["STARS"] <= 5)).count()
num_valores_entre_1_y_3 = y_reviews.filter((y_reviews["STARS"] >= 1) & (y_reviews["STARS"] <= 3)).count()
num_registros_en_cero = y_reviews.filter(y_reviews["STARS"] == 0).count()

print("Número total de registros donde 'STARS' es igual a 0:", num_registros_en_cero)
print("Número total de registros:", total_registros)
print("Número total de valores entre 4 y 5 en 'STARS':", num_valores_entre_4_y_5)
print("Número total de valores entre 1 y 3 en 'STARS':", num_valores_entre_1_y_3)

StatementMeta(, , , Waiting, )

En Google Solo el 17.9% de las reseñas son de menos de 4 estrellas, las demás reseñas son entre 4 y 5 estrellas. Esto opuesto a Yelp donde el 50.3% son de menos de 4 estrellas, lo que quieres dedcir que el usuario de Yelp suele calificar mas bajo a los establecimientos.

Las reseñas de Google Reviews son 49 veces más que las reseñas de Yelp, por ende es una muestra mas significativa.

## Descarga y lectura de TripAdvisor

In [ ]:
from azure.storage.blob import BlobServiceClient

# Crea una instancia del cliente de servicio Blob con tu cadena de conexión
blob_service_client = BlobServiceClient.from_connection_string("tu_cadena_de_conexión")

# Especifica el contenedor y el nombre del blob (archivo)
container_name = "558991ad-6fdf-463a-b58a-238ddabaa398@onelake.dfs.fabric.microsoft.com"
blob_name = "37a0382d-f783-4728-a697-48b1dc002937/Files/DATA FINAL/TRIP ADVISOR/APIs_Tripadvisor.csv"

# Obtiene el blob
blob_client = blob_service_client.get_blob_client(container_name, blob_name)

# Descarga el blob a un archivo local
with open("APIs_Tripadvisor.csv", "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

# Ahora puedes leer el archivo descargado con pandas
df = pd.read_csv('APIs_Tripadvisor.csv')


StatementMeta(, , , Waiting, )